In [1]:
import gurobipy as gb
from gurobipy import GRB
from gurobipy import quicksum
import pandas as pd
import numpy as np

In [2]:
J = 15
M = 3
P = [7,4,6,9,12,8,10,11,8,7,6,8,15,14,3]
Conflict_sets = [(2-1,5-1,8-1), (6-1,9-1), (7-1,10-1), (11-1,15-1)]

In [3]:
model = gb.Model('p1')

Academic license - for non-commercial use only - expires 2021-05-20
Using license file /Users/eason/gurobi.lic


In [4]:
x = []
for j in range(J):
    tmp = []
    for m in range(M):
        tmp.append(model.addVar(vtype=GRB.BINARY, name=f'x[{j}, {m}]'))
    x.append(tmp)

w = model.addVar(lb=0, vtype=GRB.CONTINUOUS, name='w')

In [5]:
model.setObjective(w, GRB.MINIMIZE)

In [6]:
_ = model.addConstrs(
    quicksum(x[j][m] for m in range(M)) == 1
for j in range(J))

_ = model.addConstrs(
quicksum(x[j][m] for j in con_set) <= 1
for m in range(M)
for con_set in Conflict_sets)

_ = model.addConstrs(
    quicksum(P[j] * x[j][m] for j in range(J)) <= w 
for m in range(M))

In [7]:
model.optimize()

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 30 rows, 46 columns and 120 nonzeros
Model fingerprint: 0xb569022c
Variable types: 1 continuous, 45 integer (45 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 72.0000000
Presolve time: 0.00s
Presolved: 30 rows, 46 columns, 120 nonzeros
Variable types: 0 continuous, 46 integer (45 binary)

Root relaxation: objective 4.266667e+01, 25 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   42.66667    0    5   72.00000   42.66667  40.7%     -    0s
H    0     0                      45.0000000   42.66667  5.19%     -    0s
H    0     0                 

In [8]:
for v in model.getVars():
    print(f'{v.varName}: {int(v.x)}')
print(f'Obj value: {model.ObjVal:.5f}')

x[0, 0]: 1
x[0, 1]: 0
x[0, 2]: 0
x[1, 0]: 0
x[1, 1]: 0
x[1, 2]: 1
x[2, 0]: 0
x[2, 1]: 0
x[2, 2]: 1
x[3, 0]: 0
x[3, 1]: 0
x[3, 2]: 1
x[4, 0]: 1
x[4, 1]: 0
x[4, 2]: 0
x[5, 0]: 0
x[5, 1]: 1
x[5, 2]: 0
x[6, 0]: 0
x[6, 1]: 1
x[6, 2]: 0
x[7, 0]: 0
x[7, 1]: 1
x[7, 2]: 0
x[8, 0]: 0
x[8, 1]: 0
x[8, 2]: 1
x[9, 0]: 1
x[9, 1]: 0
x[9, 2]: 0
x[10, 0]: 0
x[10, 1]: 1
x[10, 2]: 0
x[11, 0]: 0
x[11, 1]: 1
x[11, 2]: 0
x[12, 0]: 0
x[12, 1]: 0
x[12, 2]: 1
x[13, 0]: 1
x[13, 1]: 0
x[13, 2]: 0
x[14, 0]: 1
x[14, 1]: 0
x[14, 2]: 0
w: 43
Obj value: 43.00000


In [9]:
L = [
[0,	3,	4,	6,	8,	9,	8,	10],
[3,	0,	5,	4,	8,  6,	12,	9],
[4,	5,	0,	2,	2,	3,	5,	7],
[6,	4,	2,	0,	3,	2,	5,	4],
[8,	8,	2,	3,	0,	2,	2,	4],
[9,	6,	3,	2,	2,	0,	3,	2],
[8,	12,	5,	5,	2,	3,	0,	2],
[10, 9,	7,	4,	4,	2,	2,	0],
]
P = [40, 30, 35, 20, 15, 50, 45, 60]
D = 8
M = 3

In [10]:
def get_max_weighted(have_car_set):
    have_car_list = [0,0,0,0,0,0,0,0]
    for i in have_car_set: have_car_list[i] = 1
    dist = [L[i] for i in range(D) if have_car_list[i] == 1]
    dist = [min(i) for i in zip(*dist)]
    weighted = [i*j for i, j in zip(dist, P)]
    # print(weighted)
    return max(weighted)

have_car_set = set()
for m in range(3):
    try_sets = [set(have_car_set) for _ in range(D)]
    for i, _set in enumerate(try_sets): _set.add(i)
    print(try_sets)
    print(np.array([get_max_weighted(_set) for _set in try_sets]))
    index = np.argmin(np.array([get_max_weighted(_set) for _set in try_sets]))
    print(index)
    have_car_set.add(index)
print(get_max_weighted(have_car_set))


[{0}, {1}, {2}, {3}, {4}, {5}, {6}, {7}]
[600 540 420 240 320 360 360 400]
3
[{0, 3}, {1, 3}, {2, 3}, {3}, {3, 4}, {3, 5}, {3, 6}, {3, 7}]
[240 240 240 240 240 240 240 240]
0
[{0, 3}, {0, 1, 3}, {0, 2, 3}, {0, 3}, {0, 3, 4}, {0, 3, 5}, {0, 3, 6}, {0, 3, 7}]
[240 240 240 240 240 135 120 100]
7
100


In [11]:
a = [{1},{2},{3}]
for i in a:
    i.add(5)
a

[{1, 5}, {2, 5}, {3, 5}]

In [12]:
model2 = gb.Model('P2')

In [13]:
x = model2.addVars(D, vtype=GRB.BINARY, name='x')
y = model2.addVars(D, D, vtype=GRB.BINARY, name='y')
w = model2.addVars(D, vtype=GRB.CONTINUOUS, name='w')
omega = model2.addVar(vtype=GRB.CONTINUOUS, name='omega')

In [14]:
model2.setObjective(
    omega,
    GRB.MINIMIZE
)

In [15]:
_ = model2.addConstrs(
    w[i] * P[i] <= omega
for i in range(D)
for j in range(D))

_ = model2.addConstr(
    quicksum(x[i] for i in range(D)) == M
)

_ = model2.addConstrs(
    y[i,j] <= x[j]
for i in range(D)
for j in range(D))

_ = model2.addConstrs(
    quicksum(y[i,j] for j in range(D)) == 1
for i in range(D))

_ = model2.addConstrs(
    w[i] == quicksum(L[i][j] * y[i,j] for j in range(D))
for i in range(D))



In [16]:
model2.optimize()

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 145 rows, 81 columns and 392 nonzeros
Model fingerprint: 0x1d879426
Variable types: 9 continuous, 72 integer (72 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+00]
Found heuristic solution: objective 240.0000000
Presolve removed 78 rows and 22 columns
Presolve time: 0.00s
Presolved: 67 rows, 59 columns, 208 nonzeros
Variable types: 0 continuous, 59 integer (58 binary)

Root relaxation: objective 5.137139e+01, 34 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   51.37139    0   26  240.00000   51.37139  78.6%     -    0s
H    0     0                     100.0000000   51.37139  48.6%

In [17]:
for v in model2.getVars():
    print(f'{v.varName}: {int(v.x)}')
print(f'Obj value: {model2.ObjVal:.5f}')

x[0]: 1
x[1]: 0
x[2]: 1
x[3]: 0
x[4]: 0
x[5]: 0
x[6]: 0
x[7]: 1
y[0,0]: 1
y[0,1]: 0
y[0,2]: 0
y[0,3]: 0
y[0,4]: 0
y[0,5]: 0
y[0,6]: 0
y[0,7]: 0
y[1,0]: 1
y[1,1]: 0
y[1,2]: 0
y[1,3]: 0
y[1,4]: 0
y[1,5]: 0
y[1,6]: 0
y[1,7]: 0
y[2,0]: 0
y[2,1]: 0
y[2,2]: 1
y[2,3]: 0
y[2,4]: 0
y[2,5]: 0
y[2,6]: 0
y[2,7]: 0
y[3,0]: 0
y[3,1]: 0
y[3,2]: 1
y[3,3]: 0
y[3,4]: 0
y[3,5]: 0
y[3,6]: 0
y[3,7]: 0
y[4,0]: 0
y[4,1]: 0
y[4,2]: 1
y[4,3]: 0
y[4,4]: 0
y[4,5]: 0
y[4,6]: 0
y[4,7]: 0
y[5,0]: 0
y[5,1]: 0
y[5,2]: 0
y[5,3]: 0
y[5,4]: 0
y[5,5]: 0
y[5,6]: 0
y[5,7]: 1
y[6,0]: 0
y[6,1]: 0
y[6,2]: 0
y[6,3]: 0
y[6,4]: 0
y[6,5]: 0
y[6,6]: 0
y[6,7]: 1
y[7,0]: 0
y[7,1]: 0
y[7,2]: 0
y[7,3]: 0
y[7,4]: 0
y[7,5]: 0
y[7,6]: 0
y[7,7]: 1
w[0]: 0
w[1]: 3
w[2]: 0
w[3]: 2
w[4]: 2
w[5]: 2
w[6]: 2
w[7]: 0
omega: 100
Obj value: 100.00000
